In [1]:
# Importação das bibliotecas necessárias

import pandas as pd
import numpy as np
import glob
import os
import re
from datetime import datetime
import locale
import sys

In [2]:
# Definição da loja a ser utilizada

loja = 1
periodo = 2

In [3]:
# Importação das planilhas necessárias para
# gerar os documentos da 000, 0150, 0200, 1050 e 1100

planilha = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_v1.xlsx')
planilha = planilha.reset_index()
planilha['DATA'] = pd.to_datetime(planilha['DATA'], format='%Y-%m-%d')

planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,Valor Base Cálculo ICMS ST Retido Operação Anterior,Valor Complementar,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,COD_ITEM,CNPJ EMITENTE,tipo,vBCST,FONTE
0,0,35191003129126000159590006031131098680305954,2019-10-24,5405,3,100073,1,1,1,1.0,...,0.00,0.000000,0.0,0.0,0.000000,100073,NaN,grupo,0.000000,dfe
1,1,35191027849963000110550010005964131005945510,2019-10-29,1403,20,100073,0,1,1,NaN,...,63.62,71.877277,0.0,0.0,24.389510,100073,2.784996e+13,grupo,75.582468,entrada_sp
2,2,35191127849963000110550010006084971006065827,2019-11-09,1403,7,100073,0,1,1,NaN,...,63.62,71.877277,0.0,0.0,24.389510,100073,2.784996e+13,grupo,75.582468,entrada_sp
3,3,35191203129126000159590006031131212496179820,2019-12-05,5405,1,100073,1,1,1,NaN,...,0.00,0.000000,0.0,0.0,0.000000,100073,NaN,grupo,0.000000,dfe
4,4,35191227849963000110550010006364621006344734,2019-12-08,1403,36,100073,0,1,1,NaN,...,63.72,71.777277,0.0,0.0,24.389510,100073,2.784996e+13,grupo,75.582468,entrada_sp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,493477,35201203129126000159590007086090676940106578,2020-12-23,5405,3,99234,1,1,1,NaN,...,0.00,0.000000,0.0,0.0,0.000000,99234,NaN,grupo,0.000000,dfe
493478,493478,35201203129126000159590005812821041405820715,2020-12-24,5405,5,99234,1,1,1,NaN,...,0.00,0.000000,0.0,0.0,0.000000,99234,NaN,grupo,0.000000,dfe
493479,493479,35201227849963000110550010010262221010226748,2020-12-29,1403,19,99234,0,1,1,NaN,...,1.88,8.606188,0.0,0.0,2.621547,99234,2.784996e+13,grupo,5.632534,entrada_sp
493480,493480,35201203129126000159590007086090696680045137,2020-12-31,5405,4,99234,1,1,1,NaN,...,0.00,0.000000,0.0,0.0,0.000000,99234,NaN,grupo,0.000000,dfe


In [4]:
tabela_2 = pd.read_csv(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\loja_{loja}_tabela_2_1_p{periodo}_v1.csv', encoding='utf-8')
tabela_2 = tabela_2.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                           ascending=[True, True, True, True])
tabela_2

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\237983676.py:1: DtypeWarning: Columns (22,28) have mixed types. Specify dtype option on import or set low_memory=False.
  tabela_2 = pd.read_csv(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\loja_{loja}_tabela_2_1_p{periodo}_v1.csv', encoding='utf-8')


,Unnamed: 0,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,tipo,COD_ITEM,IND_OPER,SUB_TIPO,...,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Valor Base Cálculo ICMS Substituição Tributária,vBCST,FONTE
428976,428976,35200303129126000159590007086090080848810419,2020-03-06,5405,1,18,grupo,18,1,1,...,9455 - DEPAKENE 250MG 25S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,0.000000,dfe
428977,428977,35200303129126000159590007086090080848810419,2020-03-06,5405,2,18,grupo,18,1,1,...,9455 - DEPAKENE 250MG 25S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,0.000000,dfe
55922,55922,35200327849963000110550010007168391007145826,2020-03-10,1403,1,18,grupo,18,0,1,...,DEPAKENE 250MG 25'S ABB C1,7891158000942.0,UN,30049099.0,1300100.0,2.784996e+13,3.129126e+12,NaN,17.664725,entrada_sp
190821,190821,35191103129126000159590005812820376086785510,2019-11-15,5405,1,34,grupo,34,1,1,...,2176 - DEPAKENE 250MG XPE 100ML AB C,7891158000966.0,UN,30049099.0,NaN,NaN,NaN,NaN,0.000000,dfe
190822,190822,35191103129126000159590005812820376086785510,2019-11-15,5405,2,34,grupo,34,1,1,...,2176 - DEPAKENE 250MG XPE 100ML AB C,7891158000966.0,UN,30049099.0,NaN,NaN,NaN,NaN,0.000000,dfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142652,142652,35201271605265006283550010000014691010230080,2020-12-31,1403,27,901004510,outros2,99036,0,1,...,CIPROFIBRATO 100MG 30S NQ,NaN,UN,30049099.0,1300200.0,7.160527e+13,3.129126e+12,NaN,133.836705,entrada_sp
353537,353537,35200403129126000159590005812820691245204225,2020-04-29,5405,4,901004511,outros2,99234,1,1,...,51655 - ESM RISQUE V TRAT BAS COB BRI,7891182991933.0,UN,33043000.0,NaN,NaN,NaN,NaN,0.000000,dfe
391835,391835,35200803129126000159590005812820839397223248,2020-08-12,5405,1,901004511,outros2,99234,1,1,...,8400 - ESM RISQUE V TRAT BAS COB BRI,7891182991933.0,UN,33043000.0,NaN,NaN,NaN,NaN,0.000000,dfe
279218,279218,35200903129126000159590005812820867797723441,2020-09-05,5405,12,901004511,outros2,99234,1,1,...,40000 - ESM RISQUE V TRAT BAS COB BRI,7891182991933.0,UN,33043000.0,NaN,NaN,NaN,NaN,0.000000,dfe


In [5]:
planilha = planilha.merge(tabela_2[['CHV_DOC', 'NUM_ITEM', 'DESCRICAO', 'CODIGO_BARRA', 'UNIDADE', 'N C M', 'CEST',
                'CNPJ DESTINATARIO']],
            on=['CHV_DOC', 'NUM_ITEM'], how='left')

In [6]:
planilha.head()

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,CNPJ EMITENTE,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO
0,0,35191003129126000159590006031131098680305954,2019-10-24,5405,3,100073,1,1,1,1.0,...,NaN,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050.0,NaN,NaN
1,1,35191027849963000110550010005964131005945510,2019-10-29,1403,20,100073,0,1,1,NaN,...,2.784996e+13,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050.0,1300101.0,3.129126e+12
2,2,35191127849963000110550010006084971006065827,2019-11-09,1403,7,100073,0,1,1,NaN,...,2.784996e+13,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050.0,1300101.0,3.129126e+12
3,3,35191203129126000159590006031131212496179820,2019-12-05,5405,1,100073,1,1,1,NaN,...,NaN,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050.0,NaN,NaN
4,4,35191227849963000110550010006364621006344734,2019-12-08,1403,36,100073,0,1,1,NaN,...,2.784996e+13,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050.0,1300101.0,3.129126e+12


In [7]:
planilha['cod novo'] = planilha['cod novo'].astype(str)

In [8]:
planilha['DESCRICAO'] = planilha['DESCRICAO'].str.replace('"', "'")

In [9]:
planilha['N C M'] = pd.to_numeric(planilha['N C M'])
planilha['N C M'] = planilha['N C M'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CEST'] = pd.to_numeric(planilha['CEST'])
planilha['CEST'] = planilha['CEST'].astype('Int64').astype(str).replace('<NA>', np.nan)
planilha['CEST'] = planilha['CEST'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CNPJ EMITENTE'] = pd.to_numeric(planilha['CNPJ EMITENTE'])
planilha['CNPJ EMITENTE'] = planilha['CNPJ EMITENTE'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['QTD_CAT'] = planilha['QTD_CAT'].astype(str) + ',000'

planilha['COD_LEGAL_PCAT'] = pd.to_numeric(planilha['COD_LEGAL_PCAT'])
planilha['COD_LEGAL_PCAT'] = planilha['COD_LEGAL_PCAT'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['CFOP'] = pd.to_numeric(planilha['CFOP'])
planilha['CFOP'] = planilha['CFOP'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['UNIDADE'] = planilha['UNIDADE'].str.strip()

def format_number(value):
    return '{:,.2f}'.format(value).replace(',', '.')

planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,CNPJ EMITENTE,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO
0,0,35191003129126000159590006031131098680305954,2019-10-24,5405,3,100073,1,1,"1,000",1.0,...,NaN,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050,nan,NaN
1,1,35191027849963000110550010005964131005945510,2019-10-29,1403,20,100073,0,1,"1,000",NaN,...,27849963000110,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12
2,2,35191127849963000110550010006084971006065827,2019-11-09,1403,7,100073,0,1,"1,000",NaN,...,27849963000110,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12
3,3,35191203129126000159590006031131212496179820,2019-12-05,5405,1,100073,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050,nan,NaN
4,4,35191227849963000110550010006364621006344734,2019-12-08,1403,36,100073,0,1,"1,000",NaN,...,27849963000110,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,493477,35201203129126000159590007086090676940106578,2020-12-23,5405,3,99234,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933.0,UN,33043000,nan,NaN
493478,493478,35201203129126000159590005812821041405820715,2020-12-24,5405,5,99234,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933.0,UN,33043000,nan,NaN
493479,493479,35201227849963000110550010010262221010226748,2020-12-29,1403,19,99234,0,1,"1,000",NaN,...,27849963000110,grupo,5.632534,entrada_sp,ESM RISQUE V TRAT BAS COB BRIL,NaN,UN,33043000,2001200,3.129126e+12
493480,493480,35201203129126000159590007086090696680045137,2020-12-31,5405,4,99234,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,38200 - ESM RISQUE V TRAT BAS COB BRI,7891182991933.0,UN,33043000,nan,NaN


In [10]:
planilha = planilha.drop_duplicates()
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,CNPJ EMITENTE,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO
0,0,35191003129126000159590006031131098680305954,2019-10-24,5405,3,100073,1,1,"1,000",1.0,...,NaN,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050,nan,NaN
1,1,35191027849963000110550010005964131005945510,2019-10-29,1403,20,100073,0,1,"1,000",NaN,...,27849963000110,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12
2,2,35191127849963000110550010006084971006065827,2019-11-09,1403,7,100073,0,1,"1,000",NaN,...,27849963000110,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12
3,3,35191203129126000159590006031131212496179820,2019-12-05,5405,1,100073,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550.0,UN,30045050,nan,NaN
4,4,35191227849963000110550010006364621006344734,2019-12-08,1403,36,100073,0,1,"1,000",NaN,...,27849963000110,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,493477,35201203129126000159590007086090676940106578,2020-12-23,5405,3,99234,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933.0,UN,33043000,nan,NaN
493478,493478,35201203129126000159590005812821041405820715,2020-12-24,5405,5,99234,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933.0,UN,33043000,nan,NaN
493479,493479,35201227849963000110550010010262221010226748,2020-12-29,1403,19,99234,0,1,"1,000",NaN,...,27849963000110,grupo,5.632534,entrada_sp,ESM RISQUE V TRAT BAS COB BRIL,NaN,UN,33043000,2001200,3.129126e+12
493480,493480,35201203129126000159590007086090696680045137,2020-12-31,5405,4,99234,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,38200 - ESM RISQUE V TRAT BAS COB BRI,7891182991933.0,UN,33043000,nan,NaN


In [11]:
planilha['CODIGO_BARRA'] = np.where(pd.notnull(planilha['CODIGO_BARRA']), planilha['CODIGO_BARRA'].astype(str).str.replace(r'\.0$', '', regex=True),
                                   np.nan)
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,CNPJ EMITENTE,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO
0,0,35191003129126000159590006031131098680305954,2019-10-24,5405,3,100073,1,1,"1,000",1.0,...,NaN,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,nan,NaN
1,1,35191027849963000110550010005964131005945510,2019-10-29,1403,20,100073,0,1,"1,000",NaN,...,27849963000110,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12
2,2,35191127849963000110550010006084971006065827,2019-11-09,1403,7,100073,0,1,"1,000",NaN,...,27849963000110,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12
3,3,35191203129126000159590006031131212496179820,2019-12-05,5405,1,100073,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,nan,NaN
4,4,35191227849963000110550010006364621006344734,2019-12-08,1403,36,100073,0,1,"1,000",NaN,...,27849963000110,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,493477,35201203129126000159590007086090676940106578,2020-12-23,5405,3,99234,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,nan,NaN
493478,493478,35201203129126000159590005812821041405820715,2020-12-24,5405,5,99234,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,nan,NaN
493479,493479,35201227849963000110550010010262221010226748,2020-12-29,1403,19,99234,0,1,"1,000",NaN,...,27849963000110,grupo,5.632534,entrada_sp,ESM RISQUE V TRAT BAS COB BRIL,NaN,UN,33043000,2001200,3.129126e+12
493480,493480,35201203129126000159590007086090696680045137,2020-12-31,5405,4,99234,1,1,"1,000",NaN,...,NaN,grupo,0.000000,dfe,38200 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,nan,NaN


In [12]:
mes_ano = []
for date in planilha['DATA']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])

planilha['Data'] = mes_ano
planilha['Data'].unique()

array(['102019', '112019', '122019', '052020', '092020', '102020',
       '032020', '122020', '012020', '022020', '042020', '062020',
       '072020', '082020', '112020'], dtype=object)

In [13]:
planilha.replace('nan', np.nan, inplace=True)

In [14]:
planilha['DATA'].astype(str).iloc[0][8:]

'24'

In [15]:
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO,Data
0,0,35191003129126000159590006031131098680305954,2019-10-24,5405,3,100073,1,1,"1,000",1.0,...,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,NaN,NaN,102019
1,1,35191027849963000110550010005964131005945510,2019-10-29,1403,20,100073,0,1,"1,000",NaN,...,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12,102019
2,2,35191127849963000110550010006084971006065827,2019-11-09,1403,7,100073,0,1,"1,000",NaN,...,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12,112019
3,3,35191203129126000159590006031131212496179820,2019-12-05,5405,1,100073,1,1,"1,000",NaN,...,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,NaN,NaN,122019
4,4,35191227849963000110550010006364621006344734,2019-12-08,1403,36,100073,0,1,"1,000",NaN,...,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12,122019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,493477,35201203129126000159590007086090676940106578,2020-12-23,5405,3,99234,1,1,"1,000",NaN,...,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,NaN,NaN,122020
493478,493478,35201203129126000159590005812821041405820715,2020-12-24,5405,5,99234,1,1,"1,000",NaN,...,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,NaN,NaN,122020
493479,493479,35201227849963000110550010010262221010226748,2020-12-29,1403,19,99234,0,1,"1,000",NaN,...,grupo,5.632534,entrada_sp,ESM RISQUE V TRAT BAS COB BRIL,NaN,UN,33043000,2001200,3.129126e+12,122020
493480,493480,35201203129126000159590007086090696680045137,2020-12-31,5405,4,99234,1,1,"1,000",NaN,...,grupo,0.000000,dfe,38200 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,NaN,NaN,122020


In [16]:
planilha['CEST'] = planilha['CEST'].replace('0', pd.NA)

# Second, add leading '0' to values with length equal to 6
planilha.loc[planilha['CEST'].str.len() == 6, 'CEST'] = '0' + planilha['CEST']

planilha['CEST']

0             NaN
1         1300101
2         1300101
3             NaN
4         1300101
           ...   
493477        NaN
493478        NaN
493479    2001200
493480        NaN
493481        NaN
Name: CEST, Length: 493482, dtype: object

In [17]:
planilha['N C M'] = planilha['N C M'].apply(lambda x: str(x).zfill(8) if len(str(x)) < 8 else str(x))
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO,Data
0,0,35191003129126000159590006031131098680305954,2019-10-24,5405,3,100073,1,1,"1,000",1.0,...,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,NaN,NaN,102019
1,1,35191027849963000110550010005964131005945510,2019-10-29,1403,20,100073,0,1,"1,000",NaN,...,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12,102019
2,2,35191127849963000110550010006084971006065827,2019-11-09,1403,7,100073,0,1,"1,000",NaN,...,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12,112019
3,3,35191203129126000159590006031131212496179820,2019-12-05,5405,1,100073,1,1,"1,000",NaN,...,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,NaN,NaN,122019
4,4,35191227849963000110550010006364621006344734,2019-12-08,1403,36,100073,0,1,"1,000",NaN,...,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12,122019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,493477,35201203129126000159590007086090676940106578,2020-12-23,5405,3,99234,1,1,"1,000",NaN,...,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,NaN,NaN,122020
493478,493478,35201203129126000159590005812821041405820715,2020-12-24,5405,5,99234,1,1,"1,000",NaN,...,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,NaN,NaN,122020
493479,493479,35201227849963000110550010010262221010226748,2020-12-29,1403,19,99234,0,1,"1,000",NaN,...,grupo,5.632534,entrada_sp,ESM RISQUE V TRAT BAS COB BRIL,NaN,UN,33043000,2001200,3.129126e+12,122020
493480,493480,35201203129126000159590007086090696680045137,2020-12-31,5405,4,99234,1,1,"1,000",NaN,...,grupo,0.000000,dfe,38200 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,NaN,NaN,122020


In [18]:
planilha.columns

Index(['index', 'CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'cod novo', 'IND_OPER',
       'SUB_TIPO', 'QTD_CAT', 'QTD_INI', 'ICMS_INI', 'QTD_ent1_devolv_ent',
       'ICMS_TOT_0', 'ICMS_TOT_ent_unit', 'ULT_ICMS_TOT_ent_unit',
       'ICMS_TOT_1', 'qtd_saida_1_devolv_saida', 'ICMS_SAIDA_UNI',
       'ULT_ICMS_SAIDA_UNI', 'ICMS_SAIDA', 'ICMS_TOT_PCAT', 'VLR_CONF_0',
       'VLR_CONFR_UNIT', 'ULT_VLR_CONFR_UNIT', 'VLR_CONFR_1', 'QTD_SALDO',
       'ICMS_TOT_SALDO', 'VLR_RESSARCIMENTO', 'VLR_COMPLEMENTO', 'COD_LEGAL',
       'COD_LEGAL_PCAT', 'VLR_CONFR_PCAT', 'SALDO_FINAL_MES_QTD',
       'SALDO_FINAL_MES_ICMS', 'ALIQUOTA', 'VALOR',
       'Valor Base Cálculo ICMS ST Retido Operação Anterior',
       'Valor Complementar', 'Valor ICMS Substituição Tributária',
       'Valor ICMS Operação', 'ICMS_TOT', 'COD_ITEM', 'CNPJ EMITENTE', 'tipo',
       'vBCST', 'FONTE', 'DESCRICAO', 'CODIGO_BARRA', 'UNIDADE', 'N C M',
       'CEST', 'CNPJ DESTINATARIO', 'Data'],
      dtype='object')

In [19]:
planilha[planilha['CODIGO_BARRA'] == 'SEM GTIN      ']

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO,Data
1035,1035,35191103129126000159550020000027911160912313,2019-11-19,1411,1,101139,0,-1,"1,000",NaN,...,grupo,0.0,entrada_oprprp,Enant.Norest+Val Estrad Cifarm,SEM GTIN,UN,30043939,1300500,0.0,112019
1638,1638,35200703129126000159550020000032371174872398,2020-07-15,1411,1,102186,0,-1,"1,000",NaN,...,grupo,0.0,entrada_oprprp,Listerine 250ml Zero Alcool,SEM GTIN,UN,33069000,2002500,0.0,072020
2161,2161,35191103129126000159550020000027941161175012,2019-11-21,1411,2,102897,0,-1,"1,000",NaN,...,grupo,0.0,entrada_oprprp,Dipirona N.Q 1gr 10S,SEM GTIN,UN,30049069,1300201,0.0,112019
2402,2402,35200703129126000159550020000032221174534864,2020-07-09,1411,1,1032,0,-1,"1,000",NaN,...,grupo,0.0,entrada_oprprp,Allestra 75/30mg 21S Ache,SEM GTIN,UN,30043939,1300300,0.0,072020
3013,3013,35200303129126000159550020000030461168852937,2020-03-30,1411,1,10447,0,-1,"1,000",NaN,...,grupo,0.0,entrada_oprprp,Sb Consecret Ft Vermelhas 500m,SEM GTIN,UN,34013000,2802300,0.0,032020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488757,488757,35200403129126000159550020000031051170393630,2020-04-27,1411,1,96826,0,-1,"1,000",NaN,...,grupo,0.0,entrada_oprprp,Ciflogex Cereja 12S Cimed,SEM GTIN,UN,30039099,1300301,0.0,042020
489021,489021,35191203129126000159550020000028791163696772,2019-12-31,1411,1,96842,0,-1,"1,000",NaN,...,grupo,0.0,entrada_oprprp,Dipimed 20ml Medquimica,SEM GTIN,UN,30039099,1300300,0.0,122019
491070,491070,35200503129126000159550020000031221170894766,2020-05-06,1411,2,97717,0,-1,"3,000",NaN,...,grupo,0.0,entrada_oprprp,Hidroclorotiazida Cimed 25mg 3,SEM GTIN,UN,30039089,1300200,0.0,052020
493323,493323,35200103129126000159550020000029421165259532,2020-01-26,1411,1,99036,0,-1,"1,000",NaN,...,grupo,0.0,entrada_oprprp,Ciprofibrato 100mg 30S Nq,SEM GTIN,UN,30049099,1300200,0.0,012020


In [20]:
planilha['CODIGO_BARRA'] = planilha['CODIGO_BARRA'].replace('SEM GTIN      ', np.nan)
planilha

,index,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,...,tipo,vBCST,FONTE,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ DESTINATARIO,Data
0,0,35191003129126000159590006031131098680305954,2019-10-24,5405,3,100073,1,1,"1,000",1.0,...,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,NaN,NaN,102019
1,1,35191027849963000110550010005964131005945510,2019-10-29,1403,20,100073,0,1,"1,000",NaN,...,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12,102019
2,2,35191127849963000110550010006084971006065827,2019-11-09,1403,7,100073,0,1,"1,000",NaN,...,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12,112019
3,3,35191203129126000159590006031131212496179820,2019-12-05,5405,1,100073,1,1,"1,000",NaN,...,grupo,0.000000,dfe,2176 - OS CAL D 60S SANOFI,7891058011550,UN,30045050,NaN,NaN,122019
4,4,35191227849963000110550010006364621006344734,2019-12-08,1403,36,100073,0,1,"1,000",NaN,...,grupo,75.582468,entrada_sp,OS CAL D 60'S SANOFI/AVENTIS,7891058011550,UN,30045050,1300101,3.129126e+12,122019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,493477,35201203129126000159590007086090676940106578,2020-12-23,5405,3,99234,1,1,"1,000",NaN,...,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,NaN,NaN,122020
493478,493478,35201203129126000159590005812821041405820715,2020-12-24,5405,5,99234,1,1,"1,000",NaN,...,grupo,0.000000,dfe,13119 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,NaN,NaN,122020
493479,493479,35201227849963000110550010010262221010226748,2020-12-29,1403,19,99234,0,1,"1,000",NaN,...,grupo,5.632534,entrada_sp,ESM RISQUE V TRAT BAS COB BRIL,NaN,UN,33043000,2001200,3.129126e+12,122020
493480,493480,35201203129126000159590007086090696680045137,2020-12-31,5405,4,99234,1,1,"1,000",NaN,...,grupo,0.000000,dfe,38200 - ESM RISQUE V TRAT BAS COB BRI,7891182991933,UN,33043000,NaN,NaN,122020


In [21]:
list(planilha['CODIGO_BARRA'].unique())

['7891058011550',
 '7890282800688',
 nan,
 '7891024139325',
 '7890282800701',
 '7891024114865',
 '7899304803690',
 '7891058325039',
 '7896658014215',
 '7897322705415',
 '7891150018761',
 '7891150006058',
 '7891150006072',
 '7891317462031',
 '7896269900761',
 '7899304803737',
 '7899304803713',
 '7895296105040',
 '7898505091479',
 '7899304803706',
 '7899304803744',
 '7899304803720',
 '7891150018853',
 '7896317907834',
 '7891058263096',
 '7891317457341',
 '7899026457645',
 '7899026459960',
 '7899026459908',
 '7899026457621',
 '7899026459649',
 '7899026459007',
 '7899026459748',
 '7899026458949',
 '7891010973902',
 '7891010974312',
 '7896004728834',
 '7891058489038',
 '7891058000165',
 '4005808313167',
 '4005808631322',
 '4005808185719',
 '4005808662388',
 '4005808663927',
 '7891058464073',
 '7896015525583',
 '7894916503792',
 '7896714207551',
 '6502400181807',
 '7896658004902',
 '7895296213042',
 '7896094904675',
 '7896094903814',
 '7891317443931',
 '7898474840528',
 '7898474842546',
 '78

In [22]:
# Geração do dataset base para geração da 1100

cols_1100 = ['CHV_DOC', 'DATA', 'NUM_ITEM', 'IND_OPER', 'cod novo', 'CFOP','QTD_CAT', 'ICMS_TOT_PCAT', 'VLR_CONFR_PCAT', 'COD_LEGAL_PCAT', 'FONTE']

df_1100 = planilha.copy()
df_1100 = df_1100[cols_1100]
df_1100['Data'] = df_1100['DATA']
df_1100['DATA'] = df_1100['DATA'].astype(str)
df_1100['DATA'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%d%m%Y') for x in df_1100['DATA']]
#df_1100.rename(coluns={'Número Item': 'NUM_ITEM'}, inplace=True)


df_1100['QTD_FIN'] = df_1100['QTD_CAT']
df_1100['COD_REG'] = ['1100']*df_1100.shape[0]
df_1100 = df_1100[['COD_REG','CHV_DOC','Data','DATA','NUM_ITEM','IND_OPER','cod novo','CFOP','QTD_FIN',
                   'ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT', 'FONTE']]
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,cod novo,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT,FONTE
0,1100,35191003129126000159590006031131098680305954,2019-10-24,24102019,3,1,100073,5405,"1,000",NaN,12.6936,1,dfe
1,1100,35191027849963000110550010005964131005945510,2019-10-29,29102019,20,0,100073,1403,"1,000",24.389510,NaN,NaN,entrada_sp
2,1100,35191127849963000110550010006084971006065827,2019-11-09,09112019,7,0,100073,1403,"1,000",24.389510,NaN,NaN,entrada_sp
3,1100,35191203129126000159590006031131212496179820,2019-12-05,05122019,1,1,100073,5405,"1,000",NaN,12.6936,1,dfe
4,1100,35191227849963000110550010006364621006344734,2019-12-08,08122019,36,0,100073,1403,"1,000",24.389510,NaN,NaN,entrada_sp
...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,1100,35201203129126000159590007086090676940106578,2020-12-23,23122020,3,1,99234,5405,"1,000",NaN,1.1225,1,dfe
493478,1100,35201203129126000159590005812821041405820715,2020-12-24,24122020,5,1,99234,5405,"1,000",NaN,1.1225,1,dfe
493479,1100,35201227849963000110550010010262221010226748,2020-12-29,29122020,19,0,99234,1403,"1,000",2.621547,NaN,NaN,entrada_sp
493480,1100,35201203129126000159590007086090696680045137,2020-12-31,31122020,4,1,99234,5405,"1,000",NaN,1.1225,1,dfe


In [23]:
df_1100[df_1100['VLR_CONFR_PCAT'] == 0]

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,cod novo,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT,FONTE


In [24]:
df_1100['Data'] = df_1100['Data'].astype(str)
df_1100['Data'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m%Y') for x in df_1100['Data']]
df_1100['Data']

0         102019
1         102019
2         112019
3         122019
4         122019
           ...  
493477    122020
493478    122020
493479    122020
493480    122020
493481    122020
Name: Data, Length: 493482, dtype: object

In [25]:
df_1100.dtypes

COD_REG            object
CHV_DOC            object
Data               object
DATA               object
NUM_ITEM            int64
IND_OPER            int64
cod novo           object
CFOP               object
QTD_FIN            object
ICMS_TOT_PCAT     float64
VLR_CONFR_PCAT    float64
COD_LEGAL_PCAT     object
FONTE              object
dtype: object

In [26]:
df_1100['QTD_FIN'] = df_1100['QTD_FIN'].replace('nan', np.nan)
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,cod novo,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT,FONTE
0,1100,35191003129126000159590006031131098680305954,102019,24102019,3,1,100073,5405,"1,000",NaN,12.6936,1,dfe
1,1100,35191027849963000110550010005964131005945510,102019,29102019,20,0,100073,1403,"1,000",24.389510,NaN,NaN,entrada_sp
2,1100,35191127849963000110550010006084971006065827,112019,09112019,7,0,100073,1403,"1,000",24.389510,NaN,NaN,entrada_sp
3,1100,35191203129126000159590006031131212496179820,122019,05122019,1,1,100073,5405,"1,000",NaN,12.6936,1,dfe
4,1100,35191227849963000110550010006364621006344734,122019,08122019,36,0,100073,1403,"1,000",24.389510,NaN,NaN,entrada_sp
...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,1100,35201203129126000159590007086090676940106578,122020,23122020,3,1,99234,5405,"1,000",NaN,1.1225,1,dfe
493478,1100,35201203129126000159590005812821041405820715,122020,24122020,5,1,99234,5405,"1,000",NaN,1.1225,1,dfe
493479,1100,35201227849963000110550010010262221010226748,122020,29122020,19,0,99234,1403,"1,000",2.621547,NaN,NaN,entrada_sp
493480,1100,35201203129126000159590007086090696680045137,122020,31122020,4,1,99234,5405,"1,000",NaN,1.1225,1,dfe


In [27]:
# Leitura do arquivo da 1050

arquivo_1050 = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\1050_loja_{loja}_p{periodo}_v1.xlsx', dtype=str)

In [28]:
mes_ano = []
for date in arquivo_1050['REF']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])
    
arquivo_1050['DATA'] = [datetime.strptime(x, '%Y%m').strftime('%m%Y') for x in arquivo_1050['REF']]
arquivo_1050

,Unnamed: 0,cod novo,REF,QTD_INI,ICMS_INI,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,DATA
0,0,18,202003,2,54.50202304495672,1,33.87973918109171,032020
1,1,34,201911,2,14.02302505075326,2,21.9418237912535,112019
2,2,34,202011,2,21.9418237912535,3,23.94145828688025,112020
3,3,34,202012,3,23.94145828688025,3,23.95225828688025,122020
4,4,67,202010,1,0,0,0,102020
...,...,...,...,...,...,...,...,...
53776,53776,901004510,202012,0,0,3,12.4128,122020
53777,53777,901004511,202004,3,1.08553201552028,2,0.7236880103468535,042020
53778,53778,901004511,202008,2,0.7236880103468535,1,0.3618440051734267,082020
53779,53779,901004511,202009,1,0.3618440051734267,0,0,092020


In [29]:
for date in planilha['Data'].unique():
    print(date)

102019
112019
122019
052020
092020
102020
032020
122020
012020
022020
042020
062020
072020
082020
112020


In [30]:
# Definição de uma função para exportar os
# dataframes de cada documentação em um
# arquivo .txt com a formatação desejada

def dataframe_to_txt_with_crlf(data, path):
    with open(path, 'a', newline='', encoding='latin-1') as f:
        for index, row in data.iterrows():
            row_str = '|'.join(map(lambda x: '' if pd.isna(x) else str(x), row)) + '\r\n'
            f.write(row_str)

In [31]:
def format_number(value):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')  # Set your desired locale
    formatted_value = locale.format('%.2f', value, grouping=True)
    formatted_value = str(formatted_value).replace('.', '')
    return formatted_value

In [32]:
df_0150 = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Referencias\\0000_0150_P{periodo}.xlsx', sheet_name='0150', dtype=str)
df_0150 = df_0150[['LOJA', 'REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'COD_MUN', 'IE', 'DATA']]
df_0150

,LOJA,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,Loja 1,0150,FOR751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,4314902,0963630237,20190901
1,Loja 1,0150,FOR752963432,REDE GAUCHA DE CERTIFICAÇÃO DIGITAL EIRELI,1058,31725974000166,NaN,4314902,0963750070,20200901
2,Loja 1,0150,FOR752036025,UNILEVER BRASIL LTDA,1058,61068276003383,NaN,3549904,125031328110,20191101
3,Loja 1,0150,FOR752036025,UNILEVER BRASIL LTDA,1058,61068276003383,NaN,3549904,125031328110,20191201
4,Loja 1,0150,FOR752036025,UNILEVER BRASIL LTDA,1058,61068276003383,NaN,3549904,125031328110,20200101
...,...,...,...,...,...,...,...,...,...,...
161227,Loja 99,0150,FOR753009243,PATRICIA ANGELICA DOS SANTOS TORRES,1058,NaN,04435689790,3550308,NaN,20201201
161228,Loja 99,0150,FOR753009567,RICARDO APRIGIO DE OLIVEIRA,1058,NaN,04437118600,3550308,NaN,20201201
161229,Loja 99,0150,FOR753009638,ANTONIO LORENCO BARBOSA,1058,NaN,00644575832,3550308,NaN,20201201
161230,Loja 99,0150,FOR753010472,EDNOLIA GARCIA DE SOUZA,1058,NaN,60428490506,3550308,NaN,20201201


In [33]:
df_0150 = df_0150[df_0150['LOJA'] == f'Loja {loja}']
if df_0150.shape[0] == 0:
        print(f'Loja {loja} não possui 0150. Favor atualizar arquivo')
        sys.exit()
df_0150['DATA'] = df_0150['DATA'].str.slice(0, 10).astype(str)
df_0150['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0150['DATA']]
df_0150['CNPJ'] = df_0150['CNPJ'].replace('', np.nan)
df_0150['IE'] = df_0150['IE'].replace('', np.nan)
df_0150['IE'] = df_0150['IE'].str.lstrip('\n')
df_0150['COD_PART'] = df_0150['COD_PART'].str.replace(r'^(FOR|CLI)', '', regex=True)
df_0150

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1251421117.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150['DATA'] = df_0150['DATA'].str.slice(0, 10).astype(str)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1251421117.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0150['DATA']]
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1251421117.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,LOJA,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,Loja 1,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,4314902,0963630237,092019
1,Loja 1,0150,752963432,REDE GAUCHA DE CERTIFICAÇÃO DIGITAL EIRELI,1058,31725974000166,NaN,4314902,0963750070,092020
2,Loja 1,0150,752036025,UNILEVER BRASIL LTDA,1058,61068276003383,NaN,3549904,125031328110,112019
3,Loja 1,0150,752036025,UNILEVER BRASIL LTDA,1058,61068276003383,NaN,3549904,125031328110,122019
4,Loja 1,0150,752036025,UNILEVER BRASIL LTDA,1058,61068276003383,NaN,3549904,125031328110,012020
...,...,...,...,...,...,...,...,...,...,...
821,Loja 1,0150,164665,VISA,1058,88888888000000,NaN,3549904,NaN,122019
822,Loja 1,0150,164665,VISA,1058,88888888000000,NaN,3549904,NaN,012020
823,Loja 1,0150,164665,VISA,1058,88888888000000,NaN,3549904,NaN,032020
824,Loja 1,0150,164665,VISA,1058,88888888000000,NaN,3549904,NaN,042020


In [34]:
df_0150 = df_0150[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'IE', 'COD_MUN', 'DATA']]
df_0150 = df_0150[df_0150['CNPJ'].notnull()]
df_0150 = df_0150.drop_duplicates()
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,0963630237,4314902,092019
1,0150,752963432,REDE GAUCHA DE CERTIFICAÇÃO DIGITAL EIRELI,1058,31725974000166,NaN,0963750070,4314902,092020
2,0150,752036025,UNILEVER BRASIL LTDA,1058,61068276003383,NaN,125031328110,3549904,112019
3,0150,752036025,UNILEVER BRASIL LTDA,1058,61068276003383,NaN,125031328110,3549904,122019
4,0150,752036025,UNILEVER BRASIL LTDA,1058,61068276003383,NaN,125031328110,3549904,012020
...,...,...,...,...,...,...,...,...,...
821,0150,164665,VISA,1058,88888888000000,NaN,NaN,3549904,122019
822,0150,164665,VISA,1058,88888888000000,NaN,NaN,3549904,012020
823,0150,164665,VISA,1058,88888888000000,NaN,NaN,3549904,032020
824,0150,164665,VISA,1058,88888888000000,NaN,NaN,3549904,042020


In [35]:
df_0150_final = pd.DataFrame()

# Get the unique 'PERIODO' values
unique_periodos = df_0150['DATA'].unique()

# Iterate through unique 'PERIODO' values
for data_periodo in unique_periodos:
    # Filter the DataFrame for the current 'PERIODO' value
    filtered_df = df_0150[df_0150['DATA'] == data_periodo]
    
    # Drop duplicates in the 'CNPJ' column for the current 'PERIODO'
    filtered_df = filtered_df.drop_duplicates(subset='CNPJ', keep='first')
    
    # Append the filtered results to the final DataFrame
    df_0150_final = pd.concat([df_0150_final, filtered_df])

# Reset the index of the final DataFrame
df_0150_final.reset_index(drop=True, inplace=True)
df_0150_final = df_0150_final[df_0150_final['REG'] == '0150']
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,0963630237,4314902,092019
1,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,209017636112,3506003,092019
2,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,125415766116,3508504,092019
3,0150,751971821,Elektro Redes S.A,1058,02328280000197,NaN,244868522118,3509502,092019
4,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,092019
...,...,...,...,...,...,...,...,...,...
238,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,NaN,645333991113,3549904,122020
239,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
240,0150,000000001,Master Formula Farmacia de Manipulação LTDA,1058,71605265039297,NaN,701237302110,3555406,122020
241,0150,752994067,LUIZ M M DOS SANTOS GAS,1058,37537780000113,NaN,701187537110,3555406,122020


In [36]:
df_0150_final = df_0150_final.drop_duplicates()
df_0150_final[df_0150_final['IE'].notnull()]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,0963630237,4314902,092019
1,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,209017636112,3506003,092019
2,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,125415766116,3508504,092019
3,0150,751971821,Elektro Redes S.A,1058,02328280000197,NaN,244868522118,3509502,092019
4,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,092019
...,...,...,...,...,...,...,...,...,...
237,0150,000002029,PROFARMA Distribuidora de Produtos Farmaceutic...,1058,45453214001042,NaN,637117955110,3548906,122020
238,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,NaN,645333991113,3549904,122020
239,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
240,0150,000000001,Master Formula Farmacia de Manipulação LTDA,1058,71605265039297,NaN,701237302110,3555406,122020


In [37]:
df_cnpj_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = list(df_0150_final['DATA'].unique())
for date in dates:
    for cnpj in df_0150_final[df_0150_final['DATA'] == date]['CNPJ'].unique():
        if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]) > 1:
            df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]
            df_cnpj_repetido = pd.concat([df_cnpj_repetido, df])
            
if df_cnpj_repetido.shape[0] > 0:
    print('Existe repetição de CNPJ. Favor verificar')
    print(f'O CNPJ repetido é: {list(df_cnpj_repetido["CNPJ"].unique())}')
    print(f'A data em que ocorreu a repetição é {list(df_cnpj_repetido["DATA"].unique())}')
    sys.exit()
    
df_cnpj_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [38]:
df_ie_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = list(df_0150_final['DATA'].unique())
for date in dates:
    for ie in df_0150_final[df_0150_final['DATA'] == date]['IE'].unique():
        if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]) > 1:
            df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]
            df_ie_repetido = pd.concat([df_ie_repetido, df])
    
if df_ie_repetido.shape[0] > 0:
    print('Existe repetição de Inscrição Estadual. Favor verificar')
    print(f'A IE repetida é: {list(df_ie_repetido["IE"].unique())}')
    print(f'A data em que ocorreu a repetição é {list(df_ie_repetido["DATA"].unique())}')
    sys.exit()
            
df_ie_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [39]:
df_0150_final = df_0150_final.drop_duplicates(subset=('COD_PART', 'DATA'), keep='first')
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,0963630237,4314902,092019
1,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,209017636112,3506003,092019
2,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,125415766116,3508504,092019
3,0150,751971821,Elektro Redes S.A,1058,02328280000197,NaN,244868522118,3509502,092019
4,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,092019
...,...,...,...,...,...,...,...,...,...
238,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,NaN,645333991113,3549904,122020
239,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
240,0150,000000001,Master Formula Farmacia de Manipulação LTDA,1058,71605265039297,NaN,701237302110,3555406,122020
241,0150,752994067,LUIZ M M DOS SANTOS GAS,1058,37537780000113,NaN,701187537110,3555406,122020


In [40]:
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,0963630237,4314902,092019
1,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,209017636112,3506003,092019
2,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,125415766116,3508504,092019
3,0150,751971821,Elektro Redes S.A,1058,02328280000197,NaN,244868522118,3509502,092019
4,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,092019
...,...,...,...,...,...,...,...,...,...
238,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,NaN,645333991113,3549904,122020
239,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
240,0150,000000001,Master Formula Farmacia de Manipulação LTDA,1058,71605265039297,NaN,701237302110,3555406,122020
241,0150,752994067,LUIZ M M DOS SANTOS GAS,1058,37537780000113,NaN,701187537110,3555406,122020


In [41]:
for cod_part in list(df_0150_final['COD_PART'].unique()):
    for date in df_0150_final['DATA']:
        if df_0150_final[(df_0150_final['COD_PART'] == cod_part) & (df_0150_final['DATA'] == date)].shape[0] > 1:
            mensagem = 'ATENÇÃO. EXISTE CODIGO DE PARTICIPANTE REPETIDO. FAVOR CHECAR'
            print(mensagem)
            print(f'O codigo repetido é igual a {cod_part}')
            print(f'A data da repetição é igual a {date}')
            sys.exit()

In [42]:
# Iterate through each file in the folder
df_0000 = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Referencias\\0000_0150_P{periodo}.xlsx', sheet_name='0000', dtype=str)

df_0000['DATA'] = df_0000['DATA'].astype(str).str.slice(0, 8)
df_0000['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0000['DATA']]
df_0000['COD_VER'] = ['01'] * df_0000.shape[0]
df_0000['COD_FIN'] = ['00'] * df_0000.shape[0]
df_0000 = df_0000[['LOJA','REG', 'DATA', 'NOME', 'CNPJ', 'IE',
        'COD_MUN', 'COD_VER', 'COD_FIN']]

df_0000

,LOJA,REG,DATA,NOME,CNPJ,IE,COD_MUN,COD_VER,COD_FIN
0,Loja 4,0000,092019,Master Formula Farmacia de Manipulação LTDA,71605265023960,701218691115,3555406,01,00
1,Loja 4,0000,102019,Master Formula Farmacia de Manipulação LTDA,71605265023960,701218691115,3555406,01,00
2,Loja 4,0000,112019,Master Formula Farmacia de Manipulação LTDA,71605265023960,701218691115,3555406,01,00
3,Loja 4,0000,122019,Master Formula Farmacia de Manipulação LTDA,71605265023960,701218691115,3555406,01,00
4,Loja 4,0000,012020,Master Formula Farmacia de Manipulação LTDA,71605265023960,701218691115,3555406,01,00
...,...,...,...,...,...,...,...,...,...
3175,Loja 71,0000,082020,Master Formula Farmacia de Manipulação LTDA,71605265000323,145800771110,3550308,01,00
3176,Loja 71,0000,092020,Master Formula Farmacia de Manipulação LTDA,71605265000323,145800771110,3550308,01,00
3177,Loja 71,0000,102020,Master Formula Farmacia de Manipulação LTDA,71605265000323,145800771110,3550308,01,00
3178,Loja 71,0000,112020,Master Formula Farmacia de Manipulação LTDA,71605265000323,145800771110,3550308,01,00


In [43]:
for date in planilha['Data'].unique():
    print(date)

102019
112019
122019
052020
092020
102020
032020
122020
012020
022020
042020
062020
072020
082020
112020


In [44]:
df_0150_final['COD_MUN'] = pd.to_numeric(df_0150_final['COD_MUN'])
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,0963630237,4314902,092019
1,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,209017636112,3506003,092019
2,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,125415766116,3508504,092019
3,0150,751971821,Elektro Redes S.A,1058,02328280000197,NaN,244868522118,3509502,092019
4,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,092019
...,...,...,...,...,...,...,...,...,...
238,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,NaN,645333991113,3549904,122020
239,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
240,0150,000000001,Master Formula Farmacia de Manipulação LTDA,1058,71605265039297,NaN,701237302110,3555406,122020
241,0150,752994067,LUIZ M M DOS SANTOS GAS,1058,37537780000113,NaN,701187537110,3555406,122020


In [45]:
def fill_missing_ie(group):
    non_null_ie = group.dropna()['IE']
    if not non_null_ie.empty:
        group['IE'] = non_null_ie.iloc[0]
    elif group['IE'].isnull().all():
        group['IE'] = np.nan
    return group

# Group by 'CNPJ' and apply the custom function
df_0150_final_new = df_0150_final.groupby('CNPJ', group_keys=False).apply(fill_missing_ie)

df_0150_final_new

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,0963630237,4314902,092019
1,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,209017636112,3506003,092019
2,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,125415766116,3508504,092019
3,0150,751971821,Elektro Redes S.A,1058,02328280000197,NaN,244868522118,3509502,092019
4,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,092019
...,...,...,...,...,...,...,...,...,...
238,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,NaN,645333991113,3549904,122020
239,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
240,0150,000000001,Master Formula Farmacia de Manipulação LTDA,1058,71605265039297,NaN,701237302110,3555406,122020
241,0150,752994067,LUIZ M M DOS SANTOS GAS,1058,37537780000113,NaN,701187537110,3555406,122020


In [46]:
df_0150_final_new = df_0150_final_new[df_0150_final_new['IE'].notnull()]
df_0150_final_new

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,0963630237,4314902,092019
1,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,209017636112,3506003,092019
2,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,125415766116,3508504,092019
3,0150,751971821,Elektro Redes S.A,1058,02328280000197,NaN,244868522118,3509502,092019
4,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,092019
...,...,...,...,...,...,...,...,...,...
237,0150,000002029,PROFARMA Distribuidora de Produtos Farmaceutic...,1058,45453214001042,NaN,637117955110,3548906,122020
238,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,NaN,645333991113,3549904,122020
239,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
240,0150,000000001,Master Formula Farmacia de Manipulação LTDA,1058,71605265039297,NaN,701237302110,3555406,122020


In [47]:
df_0150_final_new.loc[(df_0150_final_new['CNPJ'].notnull()) & (df_0150_final_new['IE'].isnull())]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [48]:
df_0150_final_new['CNPJ'] = df_0150_final_new['CNPJ'].apply(lambda x: str(x).zfill(14) if len(str(x)) < 14 else str(x))
df_0150_final_new

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\3208294570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150_final_new['CNPJ'] = df_0150_final_new['CNPJ'].apply(lambda x: str(x).zfill(14) if len(str(x)) < 14 else str(x))


,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,0963630237,4314902,092019
1,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,209017636112,3506003,092019
2,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,125415766116,3508504,092019
3,0150,751971821,Elektro Redes S.A,1058,02328280000197,NaN,244868522118,3509502,092019
4,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,092019
...,...,...,...,...,...,...,...,...,...
237,0150,000002029,PROFARMA Distribuidora de Produtos Farmaceutic...,1058,45453214001042,NaN,637117955110,3548906,122020
238,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,NaN,645333991113,3549904,122020
239,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
240,0150,000000001,Master Formula Farmacia de Manipulação LTDA,1058,71605265039297,NaN,701237302110,3555406,122020


In [49]:
# Função para verificar o dígito verificador de um CNPJ
def verifica_digito_verificador(cnpj):
    cnpj = [int(d) for d in str(cnpj) if d.isdigit()]  # Converte o CNPJ em uma lista de números inteiros
    if len(cnpj) != 14:
        return False  # CNPJ deve ter 14 dígitos

    # Calcula o primeiro dígito verificador
    soma = 0
    peso = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    for i in range(12):
        soma += cnpj[i] * peso[i]
    resto = soma % 11
    if resto < 2:
        digito_verificador1 = 0
    else:
        digito_verificador1 = 11 - resto

    # Calcula o segundo dígito verificador
    soma = 0
    peso.insert(0, 6)
    for i in range(13):
        soma += cnpj[i] * peso[i]
    resto = soma % 11
    if resto < 2:
        digito_verificador2 = 0
    else:
        digito_verificador2 = 11 - resto

    # Verifica se os dígitos verificadores são iguais aos dígitos originais
    return cnpj[-2] == digito_verificador1 and cnpj[-1] == digito_verificador2

# Exemplo de DataFrame
df = df_0150_final_new.copy()

# Aplica a função verifica_digito_verificador à coluna 'CNPJ' do DataFrame
df['Dígito_Verificador_Válido'] = df['CNPJ'].apply(verifica_digito_verificador)

# Exibe o DataFrame com a nova coluna
df

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA,Dígito_Verificador_Válido
0,0150,751986424,REDE IDEIA SOLUCOES TECNOLOGICAS LTDA EPP,1058,10707895000152,NaN,0963630237,4314902,092019,True
1,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,209017636112,3506003,092019,True
2,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,125415766116,3508504,092019,True
3,0150,751971821,Elektro Redes S.A,1058,02328280000197,NaN,244868522118,3509502,092019,True
4,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,283341718,5002704,092019,True
...,...,...,...,...,...,...,...,...,...,...
237,0150,000002029,PROFARMA Distribuidora de Produtos Farmaceutic...,1058,45453214001042,NaN,637117955110,3548906,122020,True
238,0150,000002112,MRM Fazzini Facundes Transportes EPP,1058,97554090000102,NaN,645333991113,3549904,122020,True
239,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020,True
240,0150,000000001,Master Formula Farmacia de Manipulação LTDA,1058,71605265039297,NaN,701237302110,3555406,122020,True


In [50]:
df['Dígito_Verificador_Válido'].unique()

array([ True])

In [51]:
if len(df['Dígito_Verificador_Válido'].unique()) > 1:
    print('Existem CNPJs com dígito verificador inválido. Favor checar.')
    print(f'O CNPJ com dígito verificador inválido é {list(df["CNPJ"].unique())}')
    sys.exit()

In [52]:
def recalcular_digito_verificador(nfe_completa):
    # Verifica se a NF-e com o dígito verificador tem o tamanho correto
    if pd.isnull(nfe_completa) or len(nfe_completa) != 44:
        raise ValueError(f"Valor inválido na coluna 'CHV_DOC': {nfe_completa}")

    # Remove o último caractere (dígito verificador)
    nfe_sem_dv = nfe_completa[:-1]

    # Restante da lógica de cálculo do dígito verificador
    soma = 0
    peso = 2
    nfe_invertida = nfe_sem_dv[::-1]

    for digito in nfe_invertida:
        soma += int(digito) * peso
        peso += 1
        if peso > 9:
            peso = 2

    resto = soma % 11
    dv = 0 if resto == 0 or resto == 1 else 11 - resto

    return dv

In [53]:
# Geração dos TXTs da CAT

for date in planilha['Data'].unique():
#for date in ['112019']:
    # Geração da 0000
    df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
    df_0000_['IE'] = df_0000_['IE'].str.replace('-', '')
    df_0000_ = df_0000_.drop('LOJA', axis=1)
    if df_0000_.shape[0] != 1:
        print(f'Erro no 0000 da loja {loja} para a data {date}. Favor verificar')
        sys.exit()
        
    # Geração da 0150

    # df_0150 has originally 8 columns
    df_0150_ = df_0150_final_new[df_0150_final_new['DATA'] == date]
    if df_0150_.shape[0] == 0:
        print(f'Loja {loja} não possui 0150. Favor atualizar arquivo')
        sys.exit()
    df_0150_ = df_0150_[df_0150_['CNPJ'].notnull()]
    df_0150_['IE'] = df_0150_['IE'].str.replace('-', '')
    df_0150_ = df_0150_[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF',
                'IE', 'COD_MUN']]
    
    # Geração da 0200
    df_0200 = planilha[planilha['Data'] == date][['cod novo','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']].drop_duplicates()
    df_0200['COD_REG'] = ['0200']*(df_0200.shape[0])
    df_0200 = df_0200[['COD_REG', 'cod novo','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']]
    df_0200['ALIQUOTA'] = df_0200['ALIQUOTA'].astype(float).astype(int)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].astype(str)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].replace('nan', np.nan)
    df_0200.drop_duplicates(subset='cod novo', keep='first', inplace=True)
    for cod in list(df_0200['cod novo'].unique()):
        if df_0200[df_0200['cod novo'] == cod].shape[0] > 1:
            mensagem = 'ATENÇÃO. EXISTE CODIGO DE PRODUTO DUPLICADO NO 0200.'
            print(mensagem)
            print(f'O código duplicado é {cod}')
            sys.exit()
    # Geração da 1050
    unique_cod_items = planilha[planilha['Data'] == date]['cod novo'].unique()
    arquivo_1050['cod novo'] = arquivo_1050['cod novo'].astype(str)
    filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['cod novo'].isin(unique_cod_items)]
    filtered

    df_1050 = filtered[['cod novo','QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
    
    df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
    df_1050['QTD_INI'] = df_1050['QTD_INI'].astype(float).astype(int).astype(str) + ',000'
    df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
    df_1050['SALDO_FINAL_MES_QTD'] = df_1050['SALDO_FINAL_MES_QTD'].astype(float).astype(int).astype(str) + ',000'
    df_1050['SALDO_FINAL_MES_ICMS'] = df_1050['SALDO_FINAL_MES_ICMS'].astype(float).map(format_number)
    
    df_1050 = df_1050[['COD_REG', 'cod novo', 'QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
    df_1050.rename(columns={'COD_REG': 'REG'}, inplace=True)
    
    # Geração da 1100
    df_1100_ = df_1100[df_1100['Data'] == date]
    if df_1100_[df_1100_['VLR_CONFR_PCAT'] < 0.01].shape[0] > 0:
        mensagem = 'ATNEÇÂO. EXISTE VALOR DE CONFRONTO IGUAL A 0.'
        print(mensagem)
        sys.exit()
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_ = df_1100_[['COD_REG','CHV_DOC','DATA','NUM_ITEM','IND_OPER','cod novo','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT', 'FONTE']]

            
    cnpj_loja = df_0000[df_0000['LOJA'] == f'Loja {loja}']['CNPJ'].values[0]
    df_1100_['CHV_DOC'] = np.where(df_1100_['FONTE'].isin(['entrada_oprprp', 'saida_nfe', 'dfe']),
                           df_1100_['CHV_DOC'].str.slice(0,6) + cnpj_loja + df_1100_['CHV_DOC'].str.slice(20,44),
                           df_1100_['CHV_DOC'])
    
    df_1100_['DV_NFe'] = df_1100_['CHV_DOC'].apply(recalcular_digito_verificador)
    df_1100_['CHV_DOC'] = df_1100_['CHV_DOC'].str.slice(0,-1) + df_1100_['DV_NFe'].astype(str)
    
    df_1100_['CNPJ'] = df_1100_['CHV_DOC'].str.slice(6,20)
    for cnpj in df_1100_['CNPJ'].unique():
        if df_0150_[df_0150_['CNPJ'] == cnpj].shape[0] == 0:
            mensagem = 'ATENÇÃO. EXISTE PARTICIPANTE NO 1100 QUE NÃO ESTÁ NO 0150.'
            print(mensagem)
            print(f'O participante faltante no 0150 tem CNPJ {cnpj}, para o mês {date}')
            sys.exit()
    
    df_1100_ = df_1100_[['COD_REG','CHV_DOC','DATA','NUM_ITEM','IND_OPER','cod novo','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]
    
    # Geração do .txt da 0000
    dataframe_to_txt_with_crlf(df_0000_, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\0000_{date}_{loja}_v1.txt')
    
    # Geração do .txt da 0150
    dataframe_to_txt_with_crlf(df_0150_, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\0150_{date}_{loja}_v1.txt')
    
    # Geração do .txt da 0200
    dataframe_to_txt_with_crlf(df_0200, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\0200_{date}_{loja}_v1.txt')
    
    # Geração do .txt da 1050
    dataframe_to_txt_with_crlf(df_1050, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\1050_{date}_{loja}_v1.txt')
    
    # Geração do .txt da 1100
    dataframe_to_txt_with_crlf(df_1100_, f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\1100_{date}_{loja}_v1.txt')
    
    # Geração de um arquivo .txt único,
    # contendo todas as informações das documentações
    # da 0000, 0150, 0200, 1050 e 1100
    # dentro do formato correto

    read_files = glob.glob(f"C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\Teste_loja_{loja}\\*{date}_{loja}_v1.txt")
    
    with open(f"C:\\Bots\\Bot_transmissao\\Geracao CAT\\cat_Loja_{loja}_{date}_v1.txt", "wb") as outfile:
        for f in read_files:
            with open(f, "rb") as infile:
                outfile.write(infile.read())

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['cod novo'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:48: SettingWithCopyWarning: 
A value is trying to be set

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:65: SettingWithCopyWarning: 
A value is 

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:64: SettingWithCopyWarning: 
A value is tryi

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:64: SettingWithCopyWarning: 
A value is tryi

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:64: SettingWithCopyWarning: 
A value is tryi

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:64: SettingWithCopyWarning: 
A value is tryi

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:64: SettingWithCopyWarning: 
A value is tryi

C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_36784\1888140996.py:64: SettingWithCopyWarning: 
A value is tryi